# Pyspark Handling Missing Values 
<ul>
<li>Dropping col</li>
<li>Dropping rows  </li>
<li>Various parameter In dropping functionalities </li>
<li>Handling Missing values by mean Median mode </li>
</ul>

In [1]:
import os

In [2]:
pwd = os.getcwd()
file_path= os.path.join(pwd, 'sample-1.csv')
file_path

'c:\\Users\\Admin\\Videos\\work\\Portfolio\\10.pyspark\\sample-1.csv'

In [3]:
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder.appName('DataFrame').getOrCreate()

In [5]:
spark

In [6]:
df_spark = spark.read.csv(file_path, header=True, inferSchema=True)

In [7]:
df_spark.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- experience: integer (nullable = true)
 |-- salary: integer (nullable = true)



In [8]:
df_spark.show()

+------+----+----------+------+
|  Name| Age|experience|salary|
+------+----+----------+------+
|   ram|  10|         2|   100|
| shyam|  12|         1|   150|
|sunita|  16|         1|   120|
| krish|  11|         1|   100|
| radha|  15|         2|   180|
| sudha|  13|         3|   160|
|  bala|null|         1|    80|
|  null|  14|      null|   100|
|  null|  16|      null|  null|
+------+----+----------+------+



In [9]:
df_spark.na.drop().show()

+------+---+----------+------+
|  Name|Age|experience|salary|
+------+---+----------+------+
|   ram| 10|         2|   100|
| shyam| 12|         1|   150|
|sunita| 16|         1|   120|
| krish| 11|         1|   100|
| radha| 15|         2|   180|
| sudha| 13|         3|   160|
+------+---+----------+------+



In [10]:
#any = = how---any,  all 
df_spark.na.drop(how = 'any').show()

+------+---+----------+------+
|  Name|Age|experience|salary|
+------+---+----------+------+
|   ram| 10|         2|   100|
| shyam| 12|         1|   150|
|sunita| 16|         1|   120|
| krish| 11|         1|   100|
| radha| 15|         2|   180|
| sudha| 13|         3|   160|
+------+---+----------+------+



In [11]:
df_spark.na.drop(how = 'all').show()

+------+----+----------+------+
|  Name| Age|experience|salary|
+------+----+----------+------+
|   ram|  10|         2|   100|
| shyam|  12|         1|   150|
|sunita|  16|         1|   120|
| krish|  11|         1|   100|
| radha|  15|         2|   180|
| sudha|  13|         3|   160|
|  bala|null|         1|    80|
|  null|  14|      null|   100|
|  null|  16|      null|  null|
+------+----+----------+------+



In [12]:
df_spark.na.drop(how = 'any', thresh=2).show()
#2 non null va;ue 

+------+----+----------+------+
|  Name| Age|experience|salary|
+------+----+----------+------+
|   ram|  10|         2|   100|
| shyam|  12|         1|   150|
|sunita|  16|         1|   120|
| krish|  11|         1|   100|
| radha|  15|         2|   180|
| sudha|  13|         3|   160|
|  bala|null|         1|    80|
|  null|  14|      null|   100|
+------+----+----------+------+



In [14]:
#subset 
df_spark.na.drop(how= 'any', subset = ['experience']).show()

+------+----+----------+------+
|  Name| Age|experience|salary|
+------+----+----------+------+
|   ram|  10|         2|   100|
| shyam|  12|         1|   150|
|sunita|  16|         1|   120|
| krish|  11|         1|   100|
| radha|  15|         2|   180|
| sudha|  13|         3|   160|
|  bala|null|         1|    80|
+------+----+----------+------+



In [ ]:
# fill te missing values 

In [18]:
df_spark.na.fill("Missing Values").show()

+--------------+----+----------+------+
|          Name| Age|experience|salary|
+--------------+----+----------+------+
|           ram|  10|         2|   100|
|         shyam|  12|         1|   150|
|        sunita|  16|         1|   120|
|         krish|  11|         1|   100|
|         radha|  15|         2|   180|
|         sudha|  13|         3|   160|
|          bala|null|         1|    80|
|Missing Values|  14|      null|   100|
|Missing Values|  16|      null|  null|
+--------------+----+----------+------+



In [15]:
df_spark.na.fill("Missing Values", 'Age').show()

+------+----+----------+------+
|  Name| Age|experience|salary|
+------+----+----------+------+
|   ram|  10|         2|   100|
| shyam|  12|         1|   150|
|sunita|  16|         1|   120|
| krish|  11|         1|   100|
| radha|  15|         2|   180|
| sudha|  13|         3|   160|
|  bala|null|         1|    80|
|  null|  14|      null|   100|
|  null|  16|      null|  null|
+------+----+----------+------+



#imputer function

In [20]:
from pyspark.ml.feature import Imputer
from pyspark.ml import Pipeline

In [21]:
# Create an Imputer for each column
imputers = [
    Imputer(inputCol=col, outputCol=col + "_imputed").setStrategy("mean")
    for col in ['Age', 'experience', 'salary']
]

# Create a pipeline with the imputers
pipeline = Pipeline(stages=imputers)

# Fit the pipeline to the data and transform it
df_imputed = pipeline.fit(df_spark).transform(df_spark)

In [23]:
df_imputed.show()

+------+----+----------+------+-----------+------------------+--------------+
|  Name| Age|experience|salary|Age_imputed|experience_imputed|salary_imputed|
+------+----+----------+------+-----------+------------------+--------------+
|   ram|  10|         2|   100|         10|                 2|           100|
| shyam|  12|         1|   150|         12|                 1|           150|
|sunita|  16|         1|   120|         16|                 1|           120|
| krish|  11|         1|   100|         11|                 1|           100|
| radha|  15|         2|   180|         15|                 2|           180|
| sudha|  13|         3|   160|         13|                 3|           160|
|  bala|null|         1|    80|         13|                 1|            80|
|  null|  14|      null|   100|         14|                 1|           100|
|  null|  16|      null|  null|         16|                 1|           123|
+------+----+----------+------+-----------+------------------+--

In [24]:
# Create an Imputer for each column
imputers = [
    Imputer(inputCol=col, outputCol=col + "_imputed").setStrategy("median")
    for col in ['Age', 'experience', 'salary']
]

# Create a pipeline with the imputers
pipeline = Pipeline(stages=imputers)

# Fit the pipeline to the data and transform it
df_imputed = pipeline.fit(df_spark).transform(df_spark)

In [25]:
df_imputed.show()

+------+----+----------+------+-----------+------------------+--------------+
|  Name| Age|experience|salary|Age_imputed|experience_imputed|salary_imputed|
+------+----+----------+------+-----------+------------------+--------------+
|   ram|  10|         2|   100|         10|                 2|           100|
| shyam|  12|         1|   150|         12|                 1|           150|
|sunita|  16|         1|   120|         16|                 1|           120|
| krish|  11|         1|   100|         11|                 1|           100|
| radha|  15|         2|   180|         15|                 2|           180|
| sudha|  13|         3|   160|         13|                 3|           160|
|  bala|null|         1|    80|         13|                 1|            80|
|  null|  14|      null|   100|         14|                 1|           100|
|  null|  16|      null|  null|         16|                 1|           100|
+------+----+----------+------+-----------+------------------+--